In [1]:
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

df = pd.read_csv("C:/deep learning dataset/archive (6)/spam.csv",delimiter = ',', encoding= 'latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [2]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [3]:
x = df.v2
y = df.v1
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.15)

In [5]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequence_matrix = sequence.pad_sequences(sequences,maxlen = max_len)

In [6]:
def lstm():
    inputs = Input(name = 'inputs',shape = [max_len])
    layer = Embedding(max_words,50,input_length = max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name = 'FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name = 'out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs = inputs,outputs = layer)
    return model

In [7]:
model = lstm()
model.summary()
model.compile(loss = 'binary_crossentropy',optimizer = RMSprop(),metrics = ['accuracy'])

C:\Users\shahn\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 150, 50)             │          50,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ FC1 (Dense)                          │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ out_layer (Dense)                    │ (None, 1)                   │             257 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 96,337 (376.32 KB)

 Trainable params: 96,337 (376.32 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(sequence_matrix,y_train,batch_size = 128,epochs = 10,
         validation_split = 0.2,callbacks = [EarlyStopping(monitor = 'val_loss',min_delta = 0.0001)])

In [31]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen = max_len)

In [32]:
accr = model.evaluate(test_sequences_matrix,y_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9832 - loss: 0.0911


In [33]:
print('Test set\n Loss: {:0.3f}\n Acurracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
 Loss: 0.103
 Acurracy: 0.983


In [34]:
Testing_content = ["oh k....i'm  watching here"]

txts = tok.texts_to_sequences(Testing_content)
txts = sequence.pad_sequences(txts,maxlen = max_len)

In [35]:
preds = model.predict(txts)
print(preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[3.1546133e-05]]
